In [21]:
%matplotlib inline
from scipy import signal
from scipy.io.wavfile import write,read
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython
from numpy import convolve
from scipy.signal import hamming
from scipy.ndimage.interpolation import shift
from scipy.linalg import toeplitz,inv
from scipy.fftpack import fft,fftshift,ifft,dct
from python_speech_features import *
def get_mel_filters(N,num_filts,Fs,freq_down=300,freq_up=4000):
    up_melcoef=1125*np.log(1+freq_up/700)
    down_melcoef=1125*np.log(1+freq_down/700)
    filtbank_mel=np.linspace(down_melcoef,up_melcoef,num_filts+2)
    filtbank_hz=700*(np.exp(filtbank_mel/1125)-1)
    #print (filtbank_hz)
    filtbank_bins=np.floor((N+1)*filtbank_hz/Fs)
    print(filtbank_hz)
    #print (filtbank_bins)
    #Computing filters now
    filters=np.zeros(shape=(num_filts,N))
    for i in range(1,num_filts):
        fp=int(filtbank_bins[i-1])
        sp=int(filtbank_bins[i])
        tp=int(filtbank_bins[i+1])
        filters[i-1]=np.hstack((np.zeros(fp),np.linspace(0,1,sp-fp+1),np.linspace(1,0,tp-sp+1)[1:],np.zeros(N-tp-1))).ravel()
        #plt.plot(filters[i-1])
    return filters

def feat_ext(digit,num_wav,Fs,use_inbuilt=0):
    
    # Parameters
    N=512
    num_filts=26
    mel_filters=get_mel_filters(N//2+1,num_filts,Fs)
    t_analysis=0.01
    hop_samp=int(Fs*t_analysis)
    num_samp=hop_samp
    
    for samp_under_analysis in range(1,num_wav+1):

        filename="./Processed_Data/"+str(digit)+"/"+str(samp_under_analysis)+".wav"
        print(filename+ " Read")
        read_wav = read(filename)
        inp_wo_emph=np.array(read_wav[1],dtype='float64')
        #inp=np.array(read_wav[1],dtype='float64')
        if(use_inbuilt==1):
            mfcc_feat=mfcc(inp,samplerate=8000,winlen=0.01,winstep=0.01,numcep=13,nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,ceplifter=22,appendEnergy=True)
        
        else:
            inp = np.zeros(inp_wo_emph.size)
            a=0.97
            for k in range (inp_wo_emph.size):
                if(k==0):
                    inp[k]=inp_wo_emph[k]
                else:
                    inp[k]=inp_wo_emph[k] - a*inp_wo_emph[k-1];

        
            #plt.rcParams["figure.figsize"] = (18,10)
            #plt.plot(inp,'blue')
            num_feat=int((inp.size-num_samp)/hop_samp)
            num_coef=13
            mfcc_feat=np.zeros(shape=(num_feat,num_coef))
            st_ind=0
            #print (inp.size)
            for i in range(num_feat):
    
                end_ind=st_ind+num_samp            
                #print ("Frame number: "+str(i)+" st index "+ str(st_ind)+" end index "+ str(end_ind))
                frame=inp[st_ind:end_ind]
                #---------------------------------------------------------
                #Windowed DFT
                zero_arr=np.zeros(N-num_samp)
                zero_pd=np.append(frame,zero_arr)
                hamm_w=np.append(hamming(num_samp),np.zeros((N-num_samp)))
                s_n=zero_pd*hamm_w ## x[n]=s[n]w[n]
                S_k=fft(s_n)
                #P_k=(np.abs(S_k[0:257])*np.abs(S_k[0:257]))/num_samp
                P_k=np.square(np.abs(S_k)[0:257])/num_samp
                #---------------------------------------------------------
                #Computing MFCC
                epsilon=1e-8
                energy_fbank=[20*np.log10(np.sum(P_k*mel_filters[i])+epsilon) for i in range(num_filts)]
                ifft_samp=dct(energy_fbank)
                #---------------------------------------------------------
                # DCT
                #orig_mfcc=dct(energy_fbank)
                orig_mfcc=ifft_samp[0:13]
                #if(i>=2 and i<(num_feat-2)):
                #    delta_mfcc[i]=(mfcc[i+1]-mfcc[i-1]+2*mfcc[i+2]-2*mfcc[i-2])/10
                #else:
                #    delta_mfcc=np.zeros(13)
                
                #if(i>=4 and i<(num_feat-4)):
                #    dealt_delta_mfcc=(mfcc[i+1]-mfcc[i-1]+2*mfcc[i+2]-2*mfcc[i-2])/10
                    
                #mfcc[i]=np.append(orig_mfcc,delta_mfcc[0:13])
                mfcc_feat[i]=orig_mfcc
                #print (mfcc[i])
                st_ind=st_ind+hop_samp
        print(mfcc_feat.shape)
        filename="./Extracted_Feats/"+str(digit)+"/"+str(samp_under_analysis)
        print(filename+" Saved")
        #np.save(filename,mfcc_feat)
  
feat_ext(0,2,8000)
#feat_ext(1,64,8000)
#feat_ext(2,64,8000)
#feat_ext(3,64,8000)
#feat_ext(4,64,8000)
#feat_ext(5,64,8000)
#feat_ext(6,64,8000)
#feat_ext(7,64,8000)
#feat_ext(8,64,8000)
#feat_ext(9,64,8000)
            

[  300.           358.9915951    421.4631985    487.62010143   557.67970559
   631.87223755   710.44150532   793.64569951   881.75824185   975.0686836
  1073.88365716  1178.52788362  1289.34523992  1406.69988883  1530.97747568
  1662.58639561  1801.95913565  1949.55369595  2105.85509478  2271.37696245
  2446.66322912  2632.28991226  2828.86700953  3037.04050333  3257.49448359
  3490.95339579  3738.18442161  4000.        ]
./Processed_Data/0/1.wav Read
(53, 13)
./Extracted_Feats/0/1 Saved
./Processed_Data/0/2.wav Read
(44, 13)
./Extracted_Feats/0/2 Saved


# Now Bag of Frames

In [4]:
def validation(st_index,end_index):
    test=[]
    for digits in range(10):
        digit=[]
        for i in range(st_index,end_index):
            filename="./Extracted_Feats/"+str(digits)+"/"+str(i)+".npy"
            digit.append(np.load(filename))
        test.append(digit)
    #print("Test 0,0")
    #print(test[0][3].shape[0])
    #print ("Test")
    #print(test[9][1][0].shape)
    train_samp=[i for i in range(1,65) if i not in range(st_index,end_index)]
    train=[]
    for digits in range(10):
        digit=[]
        for i in train_samp:
            filename="./Extracted_Feats/"+str(digits)+"/"+str(i)+".npy"
            digit.append(np.load(filename))
        train.append(digit)
    #print("Train 0,0")
    #print(train[0][0].shape)
    #print ("Train")
    #print(train[9][1][10].shape)
    for digit_under_analysis in range(0,10):
        print("Testing for "+str(digit_under_analysis))
        merit=np.zeros(shape=(4,10))
        for num_samp in range(4):
            for testing_digit in range(10):
                dist=0
                count=1
                for num_tr_samp in range(len(train[testing_digit])):
                    #test[digit_under_analysis][num_samp], train[testing_digit][num_tr_samp]
                    for num_frame in range(test[digit_under_analysis][num_samp].shape[0]):
                        for num_tr_frame in range(train[testing_digit][num_tr_samp].shape[0]):
                            vec_a=test[digit_under_analysis][num_samp][num_frame]
                            vec_b=train[testing_digit][num_tr_samp][num_tr_frame]
                            #print(vec_a.shape)
                            #print(vec_b.shape)
                            #epsilon=1e-8
                            samp_dist=np.sum(np.square(vec_a-vec_b))
                            #print(samp_dist)
                            dist=dist+samp_dist
                            count=count+1
                merit[num_samp][testing_digit]=dist/count
        print(merit)
        for i in range(4):
            print("Detected Digit for sample: "+ str(i)+ " is :"+ str(np.argmin(merit[i])) )
                            
            
    
validation(41,45)

Testing for 0


KeyboardInterrupt: 